# "You *rarely* want to use DataFrame.apply"

Tom Augspurger, one of the maintainers of Python's `Pandas` library for data analysis, has an awesome series of blog posts on [writing idiomatic Pandas code](https://tomaugspurger.github.io/modern-1.html). In fact you should probably leave this site now and go read one of those blog posts, they're really good. His [post on Performance](https://tomaugspurger.github.io/modern-4-performance.html) has an especially interesting tip: 

"You rarely want to use DataFrame.apply and almost never should use it with axis=1 [which processes the DataFrame row-by-row, "across columns"]. Better to write functions that take arrays and pass those in directly..." 

In Tom's example, he has a function with `numpy` math function calls, and he shows that his function works dramatically faster when those numpy functions are passed entire columns as arguments, which can be processed as vectors. Using `.apply()`, on the other and, calls those functions on one number at a time through a loop. 

## Trying out this advice on a simple text-processing function

It certainly makes sense that the vectorized approach --passing whole DataFrame columns to a function which accepts array(s) as input-- should provide significant speedup for functions with `numpy` math function calls. We expect those can operate on arrays/vectors directly. But what about text processing?

Here, I'll take a simple text-processing function I've used with `.apply()` before, and compare its performance with a slightly modified version meant to accept whole DataFrame columns instead of single strings.

First, let's load our dataset - the Quora Duplicate Questions dataset released earlier this year.

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/quora_kaggle.csv')
df.head(3)

### A simple text-processing function, and baseline speed test

The function I'll be testing is a simple text-processing function for `tokenizing` a string - returning the string as a list of words, after doing a bit of preprocessing. 

In [ ]:
import re
from nltk.corpus import stopwords

def tokenize(text):
    ''' Accept a string, return list of words (lowercased) without punctuation or stopwords'''

    # lowercase everything
    text = text.lower()
    
    # remove punctuation (\W matches any non-alphanumeric character)
    text = re.sub("\W", " ", text)
    
    # return list of words, without stopwords (stopwords are very common words which may not convey much info)
    droplist = stopwords.words('english')
    
    return [word for word in text.split() if word not in droplist]
    
tokenize('This is a sentence. And another one with punctuation and special characters to strip!?*&^%')

It takes about 49 seconds to apply this function to all of our 'question1' questions using `.apply()`:

In [ ]:
from datetime import datetime

start = datetime.now()
df['q1_tokenized'] = df['question1'].apply(tokenize) 

print('Time elapsed: ', datetime.now() - start, '\n')
print(df[['question1', 'q1_tokenized']].head(3))

..............................................

NOTE: replace 53 seconds above with 0:00:49.371585, the middle time of three runs from my shell. times were more consistent when running from my shell, think the gc provides more comparable times to compare this with next run.

................................................

### The Vectorized Approach

Let's see if we can speed this up by modifying our `tokenize` function to accept a `Pandas Series` of strings, instead of a single string. That way we won't have to use `.apply()`.

In [ ]:
def tokenize2(text_series):
    ''' Accept a series of strings, returns list of words (lowercased) without punctuation or stopwords'''

    # lowercase everything
    text_series = text_series.str.lower()
    
    # remove punctuation (r'\W' is regex, matches any non-alphanumeric character)
    text_series = text_series.str.replace(r'\W', ' ')
    
    # return list of words, without stopwords
    sw = stopwords.words('english')
    
    return text_series.apply(lambda row: [word for word in row.split() if word not in sw])

And to measure performance of the (mostly) vectorized approach: